# 실습 3: Custom 데이터로 CNN 실습
## 1. 필요한 모듈 선언하기

In [1]:
import torch
import torch.cuda as cuda
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import RandomCrop, Resize, Compose, ToTensor, Normalize
from torchvision.datasets import ImageFolder

## 2. Device 및 seed 설정

In [2]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

torch.manual_seed(1)
if device=='cuda':
    torch.cuda.manual_seed_all(1)

cuda


## 3. 데이터셋 불러오기

In [3]:
class mydataset(Dataset):
    def __init__(self, path, transfer):
        data = ImageFolder(root=path, transform=transfer)
        self.imgs = []
        self.labels = []

        for i, (img, label) in enumerate(data):
            self.imgs.append(img)
            self.labels.append([label])

        self.length = len(self.labels)
    
    def __len__(self):
        return self.length
    
    def __getitem__(self, item):
        data = self.imgs[item]
        target = torch.Tensor(self.labels[item]).to(torch.long).squeeze()
        return data, target


In [4]:
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
transferF = Compose([Resize([256, 256]), RandomCrop([224, 224]), ToTensor(), normalize])
transferFte = Compose([Resize([224, 224]), ToTensor(), normalize])

train_dataset = mydataset('./drive/MyDrive/custom_dataset/train/', transferF)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True,)

test_dataset = mydataset('./drive/MyDrive/custom_dataset/test/', transferFte)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=100,
                                           shuffle=True)

## 4. 모델 선언

In [5]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.imagesize = (224,224,3)

        self.conv1 = nn.Conv2d(3, 10, kernel_size=5, padding=2)     #out 10,224,224
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5, padding=2)    #out 20,224,224
        self.mp1 = nn.MaxPool2d(2)                                   #out 20,112,112

        self.conv3 = nn.Conv2d(20, 20, kernel_size=3, padding=1)  # out 20,112,112
        self.conv4 = nn.Conv2d(20, 20, kernel_size=3, padding=1)  # out 20,112,112
        self.mp2 = nn.MaxPool2d(2)  # out 20,56,56

        self.fc1 = nn.Linear(20 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 4)


    def forward(self, x):
        in_size = x.size(0)

        x = F.relu(self.conv1(x))
        x = F.relu(self.mp1(self.conv2(x)))

        x = F.relu(self.conv3(x))
        x = F.relu(self.mp2(self.conv4(x)))

        x = x.view(in_size, -1)  # flatten the tensor
        x = self.fc1(x)
        x = self.fc2(x)             # batch, 4

        return x

model = Net()
model = model.to(device)

## 5. 학습 시작

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)
Best = -1

for epoch in range(1, 11):
    train_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        test_loss += criterion(output, target).item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    acc = 100. * correct / len(test_loader.dataset)
    if acc > Best:
        Best = acc
        
    print('\nTrain set {} Epoch: loss: {:.4f}'.format(epoch, loss))
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%), Best Accuracy {:.0f}%\n'.format(
        test_loss, correct, len(test_loader.dataset), acc, Best))
    print('=='*100)


Train set 1 Epoch: loss: 1.2303

Test set: Average loss: 1.2535, Accuracy: 47/100 (47%), Best Accuracy 47%


Train set 2 Epoch: loss: 0.6978

Test set: Average loss: 0.7540, Accuracy: 65/100 (65%), Best Accuracy 65%


Train set 3 Epoch: loss: 0.5322

Test set: Average loss: 0.5071, Accuracy: 84/100 (84%), Best Accuracy 84%


Train set 4 Epoch: loss: 0.4476

Test set: Average loss: 0.4469, Accuracy: 83/100 (83%), Best Accuracy 84%


Train set 5 Epoch: loss: 0.2321

Test set: Average loss: 0.3138, Accuracy: 91/100 (91%), Best Accuracy 91%


Train set 6 Epoch: loss: 0.1175

Test set: Average loss: 0.3183, Accuracy: 88/100 (88%), Best Accuracy 91%


Train set 7 Epoch: loss: 0.2899

Test set: Average loss: 0.2524, Accuracy: 93/100 (93%), Best Accuracy 93%


Train set 8 Epoch: loss: 0.1391

Test set: Average loss: 0.3288, Accuracy: 95/100 (95%), Best Accuracy 95%


Train set 9 Epoch: loss: 0.0359

Test set: Average loss: 0.4740, Accuracy: 82/100 (82%), Best Accuracy 95%


Train set 10 Epoch